<a href="https://colab.research.google.com/github/RamziRBM/lab-agent-vector-store/blob/main/lab_agent_vector_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [ ]:
!pip install chromadb langchain langchain_community langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.7 MB/s eta 0:00:0

In [ ]:
from langchain_classic.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [ ]:
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [ ]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [ ]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [ ]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [ ]:
# Import things that are needed generically
from langchain_classic.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [ ]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

/tmp/ipython-input-1834837320.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the [LangGraph documentation](https://langchain-ai.github.io/langgraph/) as well as guides for [Migrating from AgentExecutor](https://python.langchain.com/docs/how_to/migrate_agent/) and LangGraph's [Pre-built ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).
  agent = initialize_agent(


In [ ]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did biden say about ketanji brown jackson in the state of the union address?"
Observation:  Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, praising her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
Thought: I now know the final answer.
Final Answer: Biden nominated Ketanji Brown Jackson to serve on the Supreme Court and praised her as one of the nation's top legal minds.

> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': "Biden nominated Ketanji Brown Jackson to serve on the Supreme Court and praised her as one of the nation's top legal minds."}

In [ ]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 There are likely many reasons why someone might choose to use ruff over flake8. I should ask the State of Union QA System and Ruff QA System to gather more information.
Action: State of Union QA System
Action Input: "What are the benefits of using ruff over flake8?"
Observation:  I don't know.
Thought: I should also ask the Ruff QA System for more specific information about ruff.
Action: Ruff QA System
Action Input: "What are the specific advantages of using ruff?"
Observation:  Some specific advantages of using Ruff include its compatibility with Black and its ability to automatically fix its own lint violations. It also has a larger rule set than other linters like Flake8 and Pylint, and can be used alongside a type checker for more comprehensive error detection. Additionally, Ruff can be used as a formatter or linter independently, and does not require the installation of Rust.
Thought: I now know the final answer.
Final Answer: Some potentia

{'input': 'Why use ruff over flake8?',
 'output': 'Some potential reasons for choosing to use Ruff over Flake8 include its compatibility with Black, its ability to automatically fix lint violations, its larger rule set, and its ability to be used independently or alongside a type checker.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [ ]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [ ]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did biden say about ketanji brown jackson in the state of the union address?"
Observation:  Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, praising her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, praising her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [ ]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 There are likely many reasons why someone might choose to use ruff over flake8. I should ask the State of Union QA System and Ruff QA System to gather more information.
Action: State of Union QA System
Action Input: "What are the benefits of using ruff over flake8?"
Observation:  I don't know.


> Finished chain.


{'input': 'Why use ruff over flake8?', 'output': " I don't know."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [ ]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 We should check both the Ruff QA System and the State of Union QA System for this information.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks.
Thought: This is not the answer we are looking for, let's try the State of Union QA System.
Action: State of Union QA System
Action Input: Did the president mention that tool in the state of the union?
Observation: 
No, the president did not mention a specific tool in the state of the union.
Thought: We should try asking a more specific question.
Action: Ruff QA System
Action Input: What specific tool did the president mention in the state of the union?
Observation:  The president did not mention any specific tool in the state of the union.
Thought: This is not the answer we are looking for, let's try the State of Union QA System again.
Action: State o

{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'The Bipartisan Infrastructure Law.'}